In [25]:
# import packages
import numpy as np

class NeuralNetwork:
    def __init__(self, layers, learnRate=0.50):
        #list of layers
        self.layers = layers
        #the learning rate
        self.learnRate = learnRate
        #list of weights between 1 and -1
        self.weights = []
        #errors at the last fit call
        self.errors = np.ones(16)
        
        #loop through the layers (except the 2 last one)
        for i in np.arange(0, len(layers) - 2):
            # init weight matrix and add a random bias
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.weights.append(w / np.sqrt(layers[i]))
        # not adding a bias to the output layer
        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.weights.append(w / np.sqrt(layers[-2]))
    
    def sigmoid(self, x):
        return 1.0 / (1 + np.exp(-x))

    def sigmoidDeriv(self, x):
        return x * (1 - x)
    
    #Check if all the errors form the pattern are all < errorMin
    def checkErrors(self, errorMin=0.05):
        error = 0
        for el in range (len(self.errors)):
            if self.errors[el] > error:
                error = self.errors[el]
        if error < 0.05:
            return True, error
        else:
            return False, error 
    
    def fit(self, X, y, epochsMax=10000, displayUpdate=100):
        
        #add a col of 1's to make the bias trainable
        X = np.c_[X, np.ones((X.shape[0]))]
        # main loop 
        cti = 0
        for epoch in np.arange(0, epochsMax):
            breaker = self.checkErrors()
            if(breaker[0] == True): #break the loop if the errorRate reach an acceptable value
                print("[END] At: {}, error: {}".format(epoch + 1, breaker[1]))
                break
            # loop to train every datapoint
            for (x, target) in zip(X, y):
                self.fit_partial(x, target, cti)
                cti = cti + 1
                if cti == 16:
                    cti = 0
            # display info every displayUpdate
            if epoch == 0 or (epoch + 1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] loss={} | epoch={}".format(
                    loss, epoch + 1))
    
    def fit_partial(self, x, y, cti):
        
        #list of output activation by layer
        A = [np.atleast_2d(x)]
        
        # loop over the layers in the network
        for layer in np.arange(0, len(self.weights)):
            
            # sigmoid function call over . of the matrixes
            net = A[layer].dot(self.weights[layer])
            out = self.sigmoid(net)
            # once we have the net output, add it to our list of
            # activations
            A.append(out)
            
        # Back propagation
        # get difference bw prediction and real value
        error = A[-1] - y
        #add the errors in the attribute (dp)
        self.errors[cti] = error
        
        #list of deltas aka D[n] = errorOutput * sigmoidDeriv 
        D = [error * self.sigmoidDeriv(A[-1])]
        
        
        #loop over the layer reverse order (not the last two took care of it earlier)
        for layer in np.arange(len(A) - 2, 0, -1):
            #calculate delata depending of the prev layer dotted with weights then mulitplied with sigmoidDeriv = activation  
            delta = D[-1].dot(self.weights[layer].T)
            delta = delta * self.sigmoidDeriv(A[layer])
            D.append(delta)
        
        # reverses delta (bc layer order)
        D = D[::-1]
        
        #updating weights
        for layer in np.arange(0, len(self.weights)):
            
            self.weights[layer] += -self.learnRate * A[layer].T.dot(D[layer])
     
    #going forward
    def predict(self, X, addBias=True):
            p = np.atleast_2d(X)
            # check if bias needs to be added
            if addBias:
                #column of 1 in the matix (last entry)
                p = np.c_[p, np.ones((p.shape[0]))]
            # loop over our layers in the network
            for layer in np.arange(0, len(self.weights)):
                p = self.sigmoid(np.dot(p, self.weights[layer]))
            # return the predicted value
            return p
    
    def calculate_loss(self, X, targets):
        # make predictions for the input data points then compute
        # the loss
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)
        # return the loss
        return loss
    
    def __repr__(self):
        # toString Equivalent
        return "NeurNw: {}".format(
            "-".join(str(l) for l in self.layers))
    
    def printW(self):
        print(self.weights)
        
    def printErr(self):
        print(self.errors) 

In [26]:
X = [[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

y = [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1]

In [27]:
nn = NeuralNetwork([4, 4, 1])
nn.fit(np.array(X), np.array(y), epochsMax=50000)
nn.printErr()
print(nn)

[INFO] loss=30.18583976651375 | epoch=1
[INFO] loss=37.74015343607593 | epoch=100
[INFO] loss=39.493409141449426 | epoch=200
[INFO] loss=41.58475616480903 | epoch=300
[INFO] loss=44.148958689362566 | epoch=400
[INFO] loss=46.749811262514314 | epoch=500
[INFO] loss=48.55725709473228 | epoch=600
[INFO] loss=49.58991497666372 | epoch=700
[INFO] loss=50.15562006139883 | epoch=800
[INFO] loss=50.316813890679306 | epoch=900
[INFO] loss=50.60850122670768 | epoch=1000
[INFO] loss=52.12245542456728 | epoch=1100
[INFO] loss=53.33775153292097 | epoch=1200
[INFO] loss=54.168698904543774 | epoch=1300
[INFO] loss=54.772986410272026 | epoch=1400
[INFO] loss=55.2381574190714 | epoch=1500
[INFO] loss=55.611191368955645 | epoch=1600
[INFO] loss=55.91939206498773 | epoch=1700
[INFO] loss=56.17977872351547 | epoch=1800
[INFO] loss=56.403534590399005 | epoch=1900
[INFO] loss=56.59832859303713 | epoch=2000
[INFO] loss=56.76962809396586 | epoch=2100
[INFO] loss=56.92147948316626 | epoch=2200
[INFO] loss=57.0

In [21]:
for (x, target) in zip(X, y):
    #Checking prediction and printing
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    if (target == step):
        print("[INFO] data={}, expected={}, pred={:.4f}, step={}".format(
            x, target, pred, step))
    else:
        print("[ERR] data={}, expected={}, pred={:.4f}, step={}".format(
            x, target, pred, step))
    
        

[INFO] data=[0, 0, 0, 0], expected=0, pred=0.0085, step=0
[INFO] data=[0, 0, 0, 1], expected=0, pred=0.0223, step=0
[INFO] data=[0, 0, 1, 0], expected=0, pred=0.0178, step=0
[INFO] data=[0, 0, 1, 1], expected=1, pred=0.9788, step=1
[INFO] data=[0, 1, 0, 0], expected=0, pred=0.0103, step=0
[INFO] data=[0, 1, 0, 1], expected=1, pred=0.9508, step=1
[INFO] data=[0, 1, 1, 0], expected=1, pred=0.9730, step=1
[INFO] data=[0, 1, 1, 1], expected=0, pred=0.0498, step=0
[INFO] data=[1, 0, 0, 0], expected=0, pred=0.0002, step=0
[INFO] data=[1, 0, 0, 1], expected=1, pred=0.9845, step=1
[INFO] data=[1, 0, 1, 0], expected=0, pred=0.0059, step=0
[INFO] data=[1, 0, 1, 1], expected=1, pred=0.9922, step=1
[INFO] data=[1, 1, 0, 0], expected=0, pred=0.0157, step=0
[INFO] data=[1, 1, 0, 1], expected=0, pred=0.0245, step=0
[INFO] data=[1, 1, 1, 0], expected=0, pred=0.0022, step=0
[INFO] data=[1, 1, 1, 1], expected=1, pred=0.9810, step=1


In [ ]:
https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
https://www.pyimagesearch.com/2021/05/06/backpropagation-from-scratch-with-python/
    